## 0. see this article
https://medium.com/qiskit/studying-quantum-walks-on-near-term-quantum-computers-f60fd2395f04

## 1. Example: Implementing quantum walk on a hypercube <a name="example1"></a>
A hypercube is the $n$-dimensional counterpart of the $3$-dimensional cube. All nodes have degree $n$, and the hypercube has a total of $N=2^n$ nodes. We can represent the nodes in a hypercube graph by $n$-tuples of binary numbers. The binary representation of the neighbors of a node will differ by only one binary number. For example, in the 4-dimensional hypercube, the neighbors to $0000$ are $0001$, $0010$, $0100$, and $1000$. Thus, a node is connected to all nodes to which the Hamming distance is 1. The edges are also labeled. Two neighboring nodes that differ in the a:th bit are connected by the edge labeled $a$.

The Hilbert space representing a coined quantum walk on the hypercube is $\mathcal{H} = \mathcal{H}^n \otimes \mathcal{H}^{2^n}$, where $\mathcal{H}^n$ denotes the coin space and $\mathcal{H}^{2^n}$ the walker's position. The computational basis is

\begin{equation}
    \big\{ \ket{a,\vec{v}}, 0 \leq a \leq n-1, \vec{v} \in  \{(00...00), (00...01), ....., (11...11 )\} \big\}. 
\end{equation}

The value of the coin computational basis $a$, which is associated with edge $a$, decides where the walker should move. If the $a=0$, the walker will go to the node where the first binary value differs from the current node. If $a=1$, the walker will move to the node in which the second value differs from the current, et cetera.  Let $\vec{e}_a$ be an n-tuple where all binary values, except the value with index $a$, are $0$. Then the shift operator $S$ moves the walker from the state $\ket{a} \ket{\vec{v}}$ to $\ket{\vec{v} \oplus \vec{e}_a}$:

\begin{equation}
    S \ket{a} \ket{\vec{v}} = \ket{a} \ket{\vec{v} \oplus \vec{e}_a}.
\end{equation}

We use the Grover coin, $G$, for this walk. Thus, the evolution operator is

\begin{equation}
    U = SG.
\end{equation}

## 2. We will now show how we can implement a quantum walk on a 4-dimensional hypercube.
We need to implement the coin operator and the shift operator. We start by importing all necessary libraries from Qiskit. 

In [99]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import QFT
from numpy import pi
import numpy as np


The circuit will have 6 qubits, 4 that represents the position and 2 that represents the coin. As we mentioned previously, the coin is a Grover coin, which is the diffuser in Grover's algorithm. We start by implementing this.

In [115]:
from qiskit.circuit.library import MCPhaseGate, MCXGate
from qiskit.quantum_info import Operator
#mcz = MCPhaseGate(np.pi, num_ctrl_qubits=4)
#qc.append(mcz, [0, 1, 2, 3, 4])
#controls = [0, 1, 2]; target = 3
#qc.append(MCXGate(len(controls)), controls + [target]) # was qc.mct([0,1,2], 3)
from qiskit import transpile
qft_gate = QFT(4, inverse=False).to_instruction()
nRzz=transpile(qft_gate.definition,basis_gates=['rzz','u']).count_ops().get('rzz', 0)
print('num Rzz=',nRzz)

num Rzz= 6


In [101]:
# Create separate quantum registers
nqCoin=2
nqPos=1<<nqCoin
qrPos = QuantumRegister(nqPos, name="pos")  # Position register with 4 qubits
qrCoin = QuantumRegister(nqCoin, name="coin")     # Coin register with 2 qubits

# Create Grover coin diffuser
qc = QuantumCircuit(qrPos, qrCoin, name="Grover coin diffuser 1 step")
for qr in qrCoin:
    qc.h(qr) ; qc.z(qr)
qc.append( MCPhaseGate(np.pi, num_ctrl_qubits=nqCoin-1),qrCoin)
for qr in qrCoin:
    qc.h(qr) 
one_step_circuit=qc
one_step_circuit.draw(fold=120)

pos_0: ──────────────────────
                              
 pos_1: ──────────────────────
                              
 pos_2: ──────────────────────
                              
 pos_3: ──────────────────────
        ┌───┐┌───┐       ┌───┐
coin_0: ┤ H ├┤ Z ├─■─────┤ H ├
        ├───┤├───┤ │P(π) ├───┤
coin_1: ┤ H ├┤ Z ├─■─────┤ H ├
        └───┘└───┘       └───┘

Now, let us implement the shift operator. We know that the walker can only move to neighboring nodes, and all neighboring nodes differ by only one bit. We want to move the walker according to the coin, and we move the walker by applying a NOT gate to one of the node qubits. If the coin is in state 
, we move the walker to the state in which the first node qubit differ. If the coin is 
 or 
, the walker moves to the state where the second and third qubit, respectively, differ. Finally, if the Grover coin is 
, we flip the fourth qubit. We implement this with CCNOT- and NOT gates after the Grover coin. Together, they are one step of a quantum walk on a 4-dimensional hypercube.

In [102]:
# Shift operator function for 4d-hypercube
def shift_operator(qc):
    for i in range(nqPos):
        qc.x(qrCoin[0])
        if i%2==0:  qc.x(qrCoin[1])
        if nqCoin>2 and i%4==0:  qc.x(qrCoin[2])
        qrTmp = qrCoin[:] + [qrPos[i]]  
        qc.append(MCXGate(nqCoin), qrTmp)
        
assert nqCoin<4 # or fix above by adding more if-stetements
shift_operator(one_step_circuit)

one_step_gate = one_step_circuit.to_instruction() 
print( one_step_gate)
one_step_circuit.draw(fold=120)

Instruction(name='Grover coin diffuser 1 step', num_qubits=6, num_clbits=0, params=[])


┌───┐                              
 pos_0: ───────────────────────────┤ X ├──────────────────────────────
                                   └─┬─┘     ┌───┐                    
 pos_1: ─────────────────────────────┼───────┤ X ├────────────────────
                                     │       └─┬─┘     ┌───┐          
 pos_2: ─────────────────────────────┼─────────┼───────┤ X ├──────────
                                     │         │       └─┬─┘     ┌───┐
 pos_3: ─────────────────────────────┼─────────┼─────────┼───────┤ X ├
        ┌───┐┌───┐       ┌───┐┌───┐  │  ┌───┐  │  ┌───┐  │  ┌───┐└─┬─┘
coin_0: ┤ H ├┤ Z ├─■─────┤ H ├┤ X ├──■──┤ X ├──■──┤ X ├──■──┤ X ├──■──
        ├───┤├───┤ │P(π) ├───┤├───┤  │  └───┘  │  ├───┤  │  └───┘  │  
coin_1: ┤ H ├┤ Z ├─■─────┤ H ├┤ X ├──■─────────■──┤ X ├──■─────────■──
        └───┘└───┘       └───┘└───┘               └───┘

## 4. Quantum walk search algorithm <a name="qwalkalgo"></a>
We will now implement a quantum walk search algorithm that finds a marked vertex in a graph. First, we describe the algorithm, then we go through its implementation. The quantum walk search algorithm solves the problem of finding marked vertices in a graph by a quantum walk. That is, we mark some set of vertices $|M|$, start at an arbitrary node in the graph and move according to the walk until we find the marked nodes. 

The basis states in the quantum walk search algorithm have two registers, one corresponding to the current node and the other corresponding to the previous node. That is, the basis states corresponds to the edges in the graph. We denote the quantum walk based on the classical Markov chain with transition matrix $P$ by the unitary operation $W(P)$ on $\mathcal{H}$. We also define $\ket{p_x} = \sum_y \sqrt{P_{xy}}\ket{y}$ as the uniform superposition over the node $x$'s neighbors. Let $\ket{x}\ket{y}$ be a basis state. We define the basis state $\ket{x}\ket{y}$ as ''good'' if $x$ is a marked node. Otherwise, we refer to it as ''bad''. We now introduce ''good'' and ''bad'' states: 

\begin{equation}
    \ket{G} = \frac{1}{\sqrt{|M|}} \sum_{x \in M} \ket{x} \ket{p_x}, \;
    \ket{B} = \frac{1}{\sqrt{N-|M|}} \sum_{x \notin M} \ket{x} \ket{p_x},
\end{equation}

which are the superpositions over good and bad basis states. Next, let us define $\epsilon = |M|/N$ and $\theta = \arcsin(\sqrt{\epsilon})$.


In short, the algorithm consists of three steps: 


1. Set up the initial state $\ket{U} = \frac{1}{\sqrt{N}} \sum_{x} \ket{x} \ket{p_x} = \sin{\theta} \ket{G} + \cos{\theta} \ket{B}$, a uniform superposition over all edges
2. Repeat $O(1/\sqrt{\epsilon})$ times: 
    
    (a) Reflect through $\ket{B}$
    
    (b) Reflect through $\ket{U}$

3. Do a measurement in the computational basis 


We can easily implement step $1$ with Hadamard gates and the reflection through $\ket{B}$ with a phase oracle that shifts the phase of $x$ if $x$ is in the first register, and leaves the circuit unchanged otherwise. 


Step 2(b) is equivalent to finding a unitary $R(P)$ that performs the following mapping: 
\begin{align}
    \ket{U} &\mapsto \ket{U}, \: \text{and} \\
    \ket{\psi} &\mapsto -\ket{\psi}, \: \forall \ket{\psi} \text{in the span of eigenvectors of $W(P)$ that are orthogonal to $\ket{U}$}
\end{align}

To find this operator we apply phase estimation on $W(P)$. Above we defined $W(P)$ as the evolution operator for the random walk. As we saw in section 2.A, this is a unitary operator. From this follows that the eigenvalues of $W(P)$ have norm $1$. Because of this, we can write the eigenvalues of $W(P)$ on the form $e^{\pm 2i\theta_j}$. The unitary $W(P)$ has one eigenvector with corresponding eigenvalue $1$, which is $\ket{U}$. This is given by $\theta_1=0$. $R(P)$ will find this vector $\ket{U}$ by adding a register with auxiliary qubits and perform phase estimation with precision $O(1/\sqrt{\delta})$, where $\delta$ is the spectral gap of $P$. To do this, we need to apply $W(P)$ $O(1/\sqrt{\delta})$ times. Let $\ket{w}$ be an eigenvector of $W(P)$ with eigenvalue $e^{\pm 2i\theta_j}$. Assume that $\tilde{\theta_j}$ is the best approximation to $\theta_j$ given by the phase estimation. The operation $R(P)$ that performs the mappings in for $\ket{w}$ in step 2(b) is given by [4]

\begin{equation}
    \ket{w} \ket{0} \mapsto \ket{w} \ket{\tilde{\theta_j}} \mapsto (-1)^{|\tilde{\theta_j} \neq 0|} \ket{w} \ket{\tilde{\theta_j}} \mapsto (-1)^{|\tilde{\theta_j} \neq 0|} \ket{w} \ket{0}
\end{equation}


## 5.Example: Quantum walk search on 4-dimensional hypercube<a name="qiskiti"></a>


The quantum walk search algorithm makes it possible to find a marked set of nodes in $O(1/\sqrt{\epsilon})$ steps, $\epsilon =  |M|/N$, where $M$ is the number of marked nodes and $N$ is the total number of nodes. This algorithm is originally used with Szegedy quantum walks, where we use two node registers to represent the quantum state. However, a coined walk with a Grover coin is equivalent to a Szegedy quantum walk and since implementations of coined walks are less complex in general, we choose to implement the algorithm with a coined walk. We will use the 4-dimensional hypercube that we implemented in section 3.

In short, we will implement the algorithm as follows. We achieve step 1, a uniform superposition over all edges, by applying Hadamard gates to the node qubits as well as the coin qubits. For step 2(a), we implement a phase oracle. Step 2(b) is implemented by a phase estimation on one step of the quantum walk on the hypercube followed by marking all quantum states where $\theta \neq 0$. We do this by rotating an auxiliary qubit. In the last part of this step, we reverse the phase estimation. The number of theta qubits depends on the precision of $\theta$. 

Below, we implement the quantum walk search algorithm on the 4-dimensional hypercube. 

For this algorithm we will need to use the inverse of the one step gate implemented earlier. We get this by using the built in circuit function inverse().

In [103]:
one_step_circuit.inverse().draw(fold=120) 

┌───┐                            
 pos_0: ──────────────────────────────┤ X ├────────────────────────────
                            ┌───┐     └─┬─┘                            
 pos_1: ────────────────────┤ X ├───────┼──────────────────────────────
                  ┌───┐     └─┬─┘       │                              
 pos_2: ──────────┤ X ├───────┼─────────┼──────────────────────────────
        ┌───┐     └─┬─┘       │         │                              
 pos_3: ┤ X ├───────┼─────────┼─────────┼──────────────────────────────
        └─┬─┘┌───┐  │  ┌───┐  │  ┌───┐  │  ┌───┐┌───┐        ┌───┐┌───┐
coin_0: ──■──┤ X ├──■──┤ X ├──■──┤ X ├──■──┤ X ├┤ H ├─■──────┤ Z ├┤ H ├
          │  └───┘  │  ├───┤  │  └───┘  │  ├───┤├───┤ │P(-π) ├───┤├───┤
coin_1: ──■─────────■──┤ X ├──■─────────■──┤ X ├┤ H ├─■──────┤ Z ├┤ H ├
                       └───┘               └───┘└───┘        └───┘└───┘

The inversed one step gate will be used to reverse the phase estimation later. We need to make controlled gates from both the one step gate that we implemented in section 3 and its inverse. We will later use them depending on the value of the control qubit.

num Rzz= 6


In [118]:
# Make controlled gates
inv_cont_one_step = one_step_circuit.inverse().control()
inv_cont_one_step_gate = inv_cont_one_step.to_instruction()
cont_one_step = one_step_circuit.control()
cont_one_step_gate = cont_one_step.to_instruction()
#inv_cont_one_step.decompose().draw(fold=120)
nRzz=transpile(one_step_gate.definition,basis_gates=['rzz','u']).count_ops().get('rzz', 0)
print('one_step_gate nRZZ=',nRzz)
nRzz=transpile(cont_one_step_gate.definition,basis_gates=['rzz','u']).count_ops().get('rzz', 0)
print('cont_one_step_gate nRZZ=',nRzz)

one_step_gate. n RZZ= 21
cont_one_step_gate. n RZZ= 193


In [ ]:
Needs more code -- see  '2021_quantum-walk-search'